In [1]:
%serialconnect to port=/dev/tty.usbserial-0001 --baud=115200


Ready.


In [2]:
import network
import ubinascii
mac = ubinascii.hexlify(network.WLAN().config('mac'),':').decode()
print(mac)

30:c6:f7:2f:27:30


In [3]:
import commands

cmd = "hciconfig"
device_id = "hci0" 
status, output = commands.getstatusoutput(cmd)
bt_mac = output.split("{}:".format(device_id))[1].split("BD Address: ")[1].split(" ")[0].strip()

print(bt_mac)

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ImportError: no module named 'commands'


In [ ]:
from machine import Pin
from machine import Timer
from time import sleep_ms
from micropython import const
import ubluetooth

_IRQ_CENTRAL_CONNECT                 = const(1 << 0)
_IRQ_CENTRAL_DISCONNECT              = const(1 << 1)
_IRQ_GATTS_WRITE                     = const(1 << 2)
_IRQ_GATTS_READ_REQUEST              = const(1 << 3)
_IRQ_SCAN_RESULT                     = const(1 << 4)
_IRQ_SCAN_COMPLETE                   = const(1 << 5)
_IRQ_PERIPHERAL_CONNECT              = const(1 << 6)
_IRQ_PERIPHERAL_DISCONNECT           = const(1 << 7)
_IRQ_GATTC_SERVICE_RESULT            = const(1 << 8)
_IRQ_GATTC_CHARACTERISTIC_RESULT     = const(1 << 9)
_IRQ_GATTC_DESCRIPTOR_RESULT         = const(1 << 10)
_IRQ_GATTC_READ_RESULT               = const(1 << 11)
_IRQ_GATTC_WRITE_STATUS              = const(1 << 12)
_IRQ_GATTC_NOTIFY                    = const(1 << 13)
_IRQ_GATTC_INDICATE                  = const(1 << 14)

ble_msg = ""
is_ble_connected = False

class esp32ble():
    def __init__(self, name):
        self.led = Pin(2, Pin.OUT)
        self.timer1 = Timer(0)
        self.name = name
        self.ble = ubluetooth.BLE()
        self.ble.active(True)
        
        self.disconnected()
        self.ble.irq(self.ble_irq)
        self.register()
        self.advertiser()

    def connected(self):
        global is_ble_connected
        is_ble_connected = True
        self.led.value(1)
        self.timer1.deinit()
        print("Device connected!")

    def disconnected(self):
        global is_ble_connected
        is_ble_connected = False
        self.timer1.init(period=100, mode=Timer.PERIODIC, callback=lambda t: self.led.value(not self.led.value()))
        print("Device disconnected!")

    def ble_irq(self, event, data):
        if event == _IRQ_CENTRAL_CONNECT:
            # The central device is already connected to this peripheral device
            conn_handle, addr_type, addr = data
            self.connected()
        elif event == _IRQ_CENTRAL_DISCONNECT:
            # The central device has been disconnected from this peripheral device
            conn_handle, addr_type, addr = data
            self.advertiser()
            self.disconnected()
        elif event == _IRQ_GATTS_WRITE:
            # The central device has written this feature or descriptor
            conn_handle, attr_handle = data
        elif event == _IRQ_GATTS_READ_REQUEST:
            # The central device has issued a read request. Note: This is a hardware IRQ
            # Return NONE to reject the read operation
            # Note: This event does not support ESP32.
            conn_handle, attr_handle = data
        elif event == _IRQ_SCAN_RESULT:
            # The result of a scan
            addr_type, addr, connectable, rssi, adv_data = data
        elif event == _IRQ_SCAN_COMPLETE:
            # Scan duration has been completed or manually stopped
            pass
        elif event == _IRQ_PERIPHERAL_CONNECT:
            #  gap_connect() connected
            conn_handle, addr_type, addr = data
        elif event == _IRQ_PERIPHERAL_DISCONNECT:
            # Connected peripherals are disconnected
            conn_handle, addr_type, addr = data
        elif event == _IRQ_GATTC_SERVICE_RESULT:
            # Call gattc_discover_services() for each service found
            conn_handle, start_handle, end_handle, uuid = data
        elif event == _IRQ_GATTC_CHARACTERISTIC_RESULT:
            # Call gattc_discover_services() every feature found
            conn_handle, def_handle, value_handle, properties, uuid = data
        elif event == _IRQ_GATTC_DESCRIPTOR_RESULT:
            # Call gattc_discover_descriptors() every descriptor found
            conn_handle, dsc_handle, uuid = data
        elif event == _IRQ_GATTC_READ_RESULT:
            # gattc_read() completed
            conn_handle, value_handle, char_data = data
        elif event == _IRQ_GATTC_WRITE_STATUS:
            # gattc_write() completed
            conn_handle, value_handle, status = data
        elif event == _IRQ_GATTC_NOTIFY:
            # The peripheral device has issued a notification request
            conn_handle, value_handle, notify_data = data
        elif event == _IRQ_GATTC_INDICATE:
            # Peripheral equipment issues instructions
            conn_handle, value_handle, notify_data = data

    def register(self):        
        NUS_UUID = '6E400001-B5A3-F393-E0A9-E50E24DCCA9E'
        RX_UUID = '6E400002-B5A3-F393-E0A9-E50E24DCCA9E'
        TX_UUID = '6E400003-B5A3-F393-E0A9-E50E24DCCA9E'
            
        BLE_NUS = ubluetooth.UUID(NUS_UUID)
        BLE_RX = (ubluetooth.UUID(RX_UUID), ubluetooth.FLAG_WRITE)
        BLE_TX = (ubluetooth.UUID(TX_UUID), ubluetooth.FLAG_NOTIFY)
            
        BLE_UART = (BLE_NUS, (BLE_TX, BLE_RX,))
        SERVICES = (BLE_UART, )
        ((self.tx, self.rx,), ) = self.ble.gatts_register_services(SERVICES)

    def send(self, data):
        self.ble.gatts_notify(0, self.tx, data + '\n')

    def advertiser(self):
        name = bytes(self.name, 'UTF-8')
        adv_data = bytearray('\x02\x01\x02') + bytearray((len(name) + 1, 0x09)) + name
        self.ble.gap_advertise(100, adv_data)
        print(adv_data)
        print("\r\n")


ble = esp32ble("ESP32")

while True:
    if is_ble_connected:
        ble.send('Hello World!')
    sleep_ms(1000)

Device disconnected!
bytearray(b'\x02\x01\x02\x06\tESP32')


.Device connected!
lld_pdu_get_tx_flush_nb HCI packet count mismatch (0, 1)
bytearray(b'\x02\x01\x02\x06\tESP32')


Device disconnected!
.Device connected!
.........................................................................................................................................................................................................................................................................................